In [2]:
# 필수 라이브러리
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option("display.max_columns", 150)
from sklearn.model_selection import GridSearchCV


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 데이터 불러오기

In [3]:
# 전체 데이터셋
raw_data = pd.read_csv('/content/drive/MyDrive/elice_contest/raw_data')
name_rawcols = list(raw_data.columns)

In [4]:
name_rawcols

['time_stamp',
 'AmbientConditions.AmbientHumidity.U.Actual',
 'AmbientConditions.AmbientTemperature.U.Actual',
 'Machine1.RawMaterial.Property1',
 'Machine1.RawMaterial.Property2',
 'Machine1.RawMaterial.Property3',
 'Machine1.RawMaterial.Property4',
 'Machine1.RawMaterialFeederParameter.U.Actual',
 'Machine1.Zone1Temperature.C.Actual',
 'Machine1.Zone2Temperature.C.Actual',
 'Machine1.MotorAmperage.U.Actual',
 'Machine1.MotorRPM.C.Actual',
 'Machine1.MaterialPressure.U.Actual',
 'Machine1.MaterialTemperature.U.Actual',
 'Machine1.ExitZoneTemperature.C.Actual',
 'Machine2.RawMaterial.Property1',
 'Machine2.RawMaterial.Property2',
 'Machine2.RawMaterial.Property3',
 'Machine2.RawMaterial.Property4',
 'Machine2.RawMaterialFeederParameter.U.Actual',
 'Machine2.Zone1Temperature.C.Actual',
 'Machine2.Zone2Temperature.C.Actual',
 'Machine2.MotorAmperage.U.Actual',
 'Machine2.MotorRPM.C.Actual',
 'Machine2.MaterialPressure.U.Actual',
 'Machine2.MaterialTemperature.U.Actual',
 'Machine2.ExitZ

In [5]:
# 제출용 데이터
submission = pd.read_csv('/content/drive/MyDrive/elice_contest/submission_df')

## 데이터 처리

In [6]:
def my_hour(X):
    return int(X[11:13])

def my_minute(X):
    return int(X[14:16])

def my_second(X):
    return int(X[17:19])

def my_minsec(X):
    return (int(X[14:16])*60) + int(X[17:19])

def my_time(X):
    return (int(X[11:13]) * 60 * 60) + (int(X[14:16])*60) + int(X[17:19]) 

In [7]:
raw_data['hour'] = raw_data['time_stamp'].apply(my_hour)
raw_data['minute'] = raw_data['time_stamp'].apply(my_minute)
raw_data['second'] = raw_data['time_stamp'].apply(my_second)

In [8]:
raw_data['minsec'] = raw_data['time_stamp'].apply(my_minsec)

In [9]:
raw_data['time'] = raw_data['time_stamp'].apply(my_time)

In [10]:
submission['hour'] = submission['time_stamp'].apply(my_hour)
submission['minute'] = submission['time_stamp'].apply(my_minute)
submission['second'] = submission['time_stamp'].apply(my_second)

In [11]:
submission['minsec'] = submission['time_stamp'].apply(my_minsec)

In [12]:
submission['time'] = submission['time_stamp'].apply(my_time)

In [13]:
#stage 1 output별 데이터 나누기
X_st1TRAIN = raw_data[['time_stamp',
 'Machine3.MaterialTemperature.U.Actual',
 'Machine3.MotorRPM.C.Actual',
 'Machine1.MaterialTemperature.U.Actual',
                     'Machine1.MotorAmperage.U.Actual',
                     'Machine1.Zone2Temperature.C.Actual',
                     'FirstStage.CombinerOperation.Temperature2.U.Actual',
                     'AmbientConditions.AmbientTemperature.U.Actual',
                     'FirstStage.CombinerOperation.Temperature1.U.Actual',
                     'Machine1.MaterialPressure.U.Actual',
                     'Machine3.MaterialPressure.U.Actual',
                      'Machine3.MotorAmperage.U.Actual',
                       #'Machine1.MotorRPM.C.Actual',
                     #'Machine1.RawMaterialFeederParameter.U.Actual',
                       #'Machine2.MaterialPressure.U.Actual',
                       'FirstStage.CombinerOperation.Temperature3.C.Actual',
                      #'Machine2.RawMaterialFeederParameter.U.Actual'
                       #'Machine3.RawMaterialFeederParameter.U.Actual',
                       'Machine2.ExitZoneTemperature.C.Actual',
                       'Machine5.ExitTemperature.U.Actual',
                       #'Machine2.MotorAmperage.U.Actual',
                       #'Machine2.Zone2Temperature.C.Actual',
                       'minute',
                       'second',
                       'minsec',
                       'time',
                       'Machine4.Temperature1.C.Actual',
                       'Machine4.Temperature2.C.Actual',
                       'Machine4.Pressure.C.Actual',
                       'Machine4.Temperature3.C.Actual',
                       'Machine4.Temperature4.C.Actual',
                       'Machine4.Temperature5.C.Actual',
                       'Machine4.ExitTemperature.U.Actual',
                       'Machine5.Temperature1.C.Actual',
                       'Machine5.Temperature2.C.Actual',
                       'Machine5.Temperature4.C.Actual',
                       'Machine5.Temperature5.C.Actual',
                       #'Machine5.Temperature3.C.Actual',
                       
                       
                      ]]

y_st1TRAIN = raw_data[['Stage1.Output.Measurement0.U.Actual',
 'Stage1.Output.Measurement1.U.Actual',
 'Stage1.Output.Measurement2.U.Actual',
 'Stage1.Output.Measurement3.U.Actual',
 'Stage1.Output.Measurement4.U.Actual',
 'Stage1.Output.Measurement5.U.Actual',
 'Stage1.Output.Measurement6.U.Actual',
 'Stage1.Output.Measurement7.U.Actual',
 'Stage1.Output.Measurement8.U.Actual',
 'Stage1.Output.Measurement9.U.Actual',
 'Stage1.Output.Measurement10.U.Actual',
 'Stage1.Output.Measurement11.U.Actual',
 'Stage1.Output.Measurement12.U.Actual',
 'Stage1.Output.Measurement13.U.Actual',
 'Stage1.Output.Measurement14.U.Actual']]


In [14]:
X_st1TEST = submission[['time_stamp',
 'Machine3.MaterialTemperature.U.Actual',
 'Machine3.MotorRPM.C.Actual',
 'Machine1.MaterialTemperature.U.Actual',
                     'Machine1.MotorAmperage.U.Actual',
                     'Machine1.Zone2Temperature.C.Actual',
                     'FirstStage.CombinerOperation.Temperature2.U.Actual',
                     'AmbientConditions.AmbientTemperature.U.Actual',
                     'FirstStage.CombinerOperation.Temperature1.U.Actual',
                     'Machine1.MaterialPressure.U.Actual',
                     'Machine3.MaterialPressure.U.Actual',
                       'Machine3.MotorAmperage.U.Actual',
                        #'Machine1.MotorRPM.C.Actual',
                        #'Machine1.RawMaterialFeederParameter.U.Actual',
                        #'Machine2.MaterialPressure.U.Actual',
                        'FirstStage.CombinerOperation.Temperature3.C.Actual',
                       #'Machine2.RawMaterialFeederParameter.U.Actual'
                        #'Machine3.RawMaterialFeederParameter.U.Actual',
                        'Machine2.ExitZoneTemperature.C.Actual',
                        'Machine5.ExitTemperature.U.Actual',
                        #'Machine2.MotorAmperage.U.Actual',
                        #'Machine2.Zone2Temperature.C.Actual',
                       'minute',
                       'second',
                        'minsec',
                        'time',
                        'Machine4.Temperature1.C.Actual',
                       'Machine4.Temperature2.C.Actual',
                       'Machine4.Pressure.C.Actual',
                       'Machine4.Temperature3.C.Actual',
                       'Machine4.Temperature4.C.Actual',
                       'Machine4.Temperature5.C.Actual',
                       'Machine4.ExitTemperature.U.Actual',
                       'Machine5.Temperature1.C.Actual',
                       'Machine5.Temperature2.C.Actual',
                       'Machine5.Temperature4.C.Actual',
                       'Machine5.Temperature5.C.Actual'
                       #'Machine5.Temperature3.C.Actual'
                       ]]

y_st1TEST = submission[[
 'Stage1.Output.Measurement0.U.Actual',
 'Stage1.Output.Measurement1.U.Actual',
 'Stage1.Output.Measurement2.U.Actual',
 'Stage1.Output.Measurement3.U.Actual',
 'Stage1.Output.Measurement4.U.Actual',
 'Stage1.Output.Measurement5.U.Actual',
 'Stage1.Output.Measurement6.U.Actual',
 'Stage1.Output.Measurement7.U.Actual',
 'Stage1.Output.Measurement8.U.Actual',
 'Stage1.Output.Measurement9.U.Actual',
 'Stage1.Output.Measurement10.U.Actual',
 'Stage1.Output.Measurement11.U.Actual',
 'Stage1.Output.Measurement12.U.Actual',
 'Stage1.Output.Measurement13.U.Actual',
 'Stage1.Output.Measurement14.U.Actual']]



In [15]:
X_st1TRAIN['time_stamp'] = pd.to_datetime(X_st1TRAIN['time_stamp'])
X_st1TRAIN= X_st1TRAIN.set_index('time_stamp')

X_st1TEST['time_stamp'] = pd.to_datetime(X_st1TEST['time_stamp'])
X_st1TEST= X_st1TEST.set_index('time_stamp')

<ipython-input-15-716cc22d98b2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_st1TRAIN['time_stamp'] = pd.to_datetime(X_st1TRAIN['time_stamp'])
<ipython-input-15-716cc22d98b2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_st1TEST['time_stamp'] = pd.to_datetime(X_st1TEST['time_stamp'])


## 제출을 위한 최종 예측값

- 반드시 numpy array로 저장해야 합니다.

In [16]:
from sklearn.ensemble import ExtraTreesRegressor

fit_reg6 = ExtraTreesRegressor(n_estimators = 1500,
                              max_depth = 38, #32
                              min_samples_split = 2,
                              random_state = 123).fit(X_st1TRAIN, y_st1TRAIN)


pred_reg6 = fit_reg6.predict(X_st1TEST)

# Station 2

In [17]:
X_st2TRAIN = raw_data[['time_stamp',
 'Machine3.MaterialTemperature.U.Actual',
                     'Machine1.MotorRPM.C.Actual',
                     'Machine3.MotorRPM.C.Actual',
                     'Machine3.MaterialPressure.U.Actual',
                     'Machine5.ExitTemperature.U.Actual',
                     'Machine1.MaterialTemperature.U.Actual',
                     'FirstStage.CombinerOperation.Temperature2.U.Actual',
                     #'Machine1.RawMaterialFeederParameter.U.Actual',
                       'Machine1.Zone2Temperature.C.Actual', #살려두기
                    #'Machine1.MaterialPressure.U.Actual',
                       'Machine3.MotorAmperage.U.Actual', #살려두기
                    #'Machine2.RawMaterialFeederParameter.U.Actual', 
                       #'Machine2.RawMaterialFeederParameter.U.Actual', 
                       'Machine2.MaterialPressure.U.Actual', 
                     'Machine5.Temperature6.C.Actual',
                     #'Machine5.Temperature3.C.Actual',
                     'Machine4.ExitTemperature.U.Actual',
                       #'Machine5.Temperature4.C.Actual',
                       #'Machine4.Temperature2.C.Actual',
                       'Machine4.Temperature5.C.Actual', #살려두기.
                       'Machine4.Temperature3.C.Actual', #살려두기.
                       #'AmbientConditions.AmbientTemperature.U.Actual', #살려두기.
                       #'Machine4.Temperature1.C.Actual',
                       'minute',
                       'second',
                       'minsec',
                       'time',
                       'Stage1.Output.Measurement0.U.Actual',
                       #'Stage1.Output.Measurement1.U.Actual',
                       'Stage1.Output.Measurement2.U.Actual',
                       'Stage1.Output.Measurement3.U.Actual',
                       'Stage1.Output.Measurement4.U.Actual',
                       #'Stage1.Output.Measurement5.U.Actual',
                       'Stage1.Output.Measurement6.U.Actual',
                       #'Stage1.Output.Measurement7.U.Actual',
                       'Stage1.Output.Measurement8.U.Actual',
                       'Stage1.Output.Measurement9.U.Actual',
                       #'Stage1.Output.Measurement10.U.Actual',
                       #'Stage1.Output.Measurement11.U.Actual',
                       #'Stage1.Output.Measurement12.U.Actual',
                       'Stage1.Output.Measurement13.U.Actual',
                       #'Stage1.Output.Measurement14.U.Actual'
                       
                       
                      ]]

y_st2TRAIN = raw_data[[
 'Stage2.Output.Measurement0.U.Actual',
 'Stage2.Output.Measurement1.U.Actual',
 'Stage2.Output.Measurement2.U.Actual',
 'Stage2.Output.Measurement3.U.Actual',
 'Stage2.Output.Measurement4.U.Actual',
 'Stage2.Output.Measurement5.U.Actual',
 'Stage2.Output.Measurement6.U.Actual',
 'Stage2.Output.Measurement7.U.Actual',
 'Stage2.Output.Measurement8.U.Actual',
 'Stage2.Output.Measurement9.U.Actual',
 'Stage2.Output.Measurement10.U.Actual',
 'Stage2.Output.Measurement11.U.Actual',
 'Stage2.Output.Measurement12.U.Actual',
 'Stage2.Output.Measurement13.U.Actual',
 'Stage2.Output.Measurement14.U.Actual']]

In [18]:
X_st2TEST = submission[['time_stamp',
 'Machine3.MaterialTemperature.U.Actual',
                     'Machine1.MotorRPM.C.Actual',
                     'Machine3.MotorRPM.C.Actual',
                     'Machine3.MaterialPressure.U.Actual',
                     'Machine5.ExitTemperature.U.Actual',
                     'Machine1.MaterialTemperature.U.Actual',
                     'FirstStage.CombinerOperation.Temperature2.U.Actual',
                     #'Machine1.RawMaterialFeederParameter.U.Actual',
                        'Machine1.Zone2Temperature.C.Actual', #살려두기.
                    #'Machine1.MaterialPressure.U.Actual', 
                        'Machine3.MotorAmperage.U.Actual', #살려두기
                    #'Machine2.RawMaterialFeederParameter.U.Actual',
                    #'Machine2.RawMaterialFeederParameter.U.Actual', 
                        'Machine2.MaterialPressure.U.Actual',
                     'Machine5.Temperature6.C.Actual',
                     #'Machine5.Temperature3.C.Actual',
                     'Machine4.ExitTemperature.U.Actual',
                        #'Machine5.Temperature4.C.Actual',
                        #'Machine4.Temperature2.C.Actual',
                        'Machine4.Temperature5.C.Actual', #살려두기
                        'Machine4.Temperature3.C.Actual', #살려두기
                        #'AmbientConditions.AmbientTemperature.U.Actual', #살려두기
                        #'Machine4.Temperature1.C.Actual',
                        'minute',
                        'second',
                        'minsec',
                        'time',
                        
                       ]]

y_st2TEST = submission[[
 'Stage2.Output.Measurement0.U.Actual',
 'Stage2.Output.Measurement1.U.Actual',
 'Stage2.Output.Measurement2.U.Actual',
 'Stage2.Output.Measurement3.U.Actual',
 'Stage2.Output.Measurement4.U.Actual',
 'Stage2.Output.Measurement5.U.Actual',
 'Stage2.Output.Measurement6.U.Actual',
 'Stage2.Output.Measurement7.U.Actual',
 'Stage2.Output.Measurement8.U.Actual',
 'Stage2.Output.Measurement9.U.Actual',
 'Stage2.Output.Measurement10.U.Actual',
 'Stage2.Output.Measurement11.U.Actual',
 'Stage2.Output.Measurement12.U.Actual',
 'Stage2.Output.Measurement13.U.Actual',
 'Stage2.Output.Measurement14.U.Actual']]

In [19]:
tmp_df = pd.DataFrame(pred_reg6)

X_st2TEST['Stage1.Output.Measurement0.U.Actual'] = tmp_df[0]
#X_st2TEST['Stage1.Output.Measurement1.U.Actual'] = tmp_df[1]
X_st2TEST['Stage1.Output.Measurement2.U.Actual'] = tmp_df[2]
X_st2TEST['Stage1.Output.Measurement3.U.Actual'] = tmp_df[3]
X_st2TEST['Stage1.Output.Measurement4.U.Actual'] = tmp_df[4]
#X_st2TEST['Stage1.Output.Measurement5.U.Actual'] = tmp_df[5]
X_st2TEST['Stage1.Output.Measurement6.U.Actual'] = tmp_df[6]
#X_st2TEST['Stage1.Output.Measurement7.U.Actual'] = tmp_df[7]
X_st2TEST['Stage1.Output.Measurement8.U.Actual'] = tmp_df[8]
X_st2TEST['Stage1.Output.Measurement9.U.Actual'] = tmp_df[9]
#X_st2TEST['Stage1.Output.Measurement10.U.Actual'] = tmp_df[10]
#X_st2TEST['Stage1.Output.Measurement11.U.Actual'] = tmp_df[11]
#X_st2TEST['Stage1.Output.Measurement12.U.Actual'] = tmp_df[12]
X_st2TEST['Stage1.Output.Measurement13.U.Actual'] = tmp_df[13]
#X_st2TEST['Stage1.Output.Measurement14.U.Actual'] = tmp_df[14]

<ipython-input-19-f4c7e7ee21e6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_st2TEST['Stage1.Output.Measurement0.U.Actual'] = tmp_df[0]
<ipython-input-19-f4c7e7ee21e6>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_st2TEST['Stage1.Output.Measurement2.U.Actual'] = tmp_df[2]
<ipython-input-19-f4c7e7ee21e6>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

In [20]:
X_st2TRAIN['time_stamp'] = pd.to_datetime(X_st2TRAIN['time_stamp'])
X_st2TRAIN= X_st2TRAIN.set_index('time_stamp')

X_st2TEST['time_stamp'] = pd.to_datetime(X_st2TEST['time_stamp'])
X_st2TEST= X_st2TEST.set_index('time_stamp')

<ipython-input-20-588e61321951>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_st2TRAIN['time_stamp'] = pd.to_datetime(X_st2TRAIN['time_stamp'])
<ipython-input-20-588e61321951>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_st2TEST['time_stamp'] = pd.to_datetime(X_st2TEST['time_stamp'])


In [ ]:
from sklearn.ensemble import ExtraTreesRegressor

fit_reg2 = ExtraTreesRegressor(n_estimators = 1500,
                              max_depth = 32,
                              min_samples_split = 2,
                              random_state = 123).fit(X_st2TRAIN, y_st2TRAIN)
pred_reg2 = fit_reg2.predict(X_st2TEST)

In [ ]:
stage1_sub_pred = pred_reg6 # stage1의 예측값
stage2_sub_pred = pred_reg2 # stage2의 예측값

## 채점

결과로 제출할 파일을 저장 후, 아래 코드를 실행하면 채점을 수행합니다.

**아래 코드를 수정하면 정상적으로 채점이 이루어지지 않을 수 있습니다.**

### 주의: 코드 파일도 채점 코드로 전달하기 때문에 아래 코드를 실행하기 전에 반드시 저장 버튼 혹은 <kbd>Ctrl</kbd> + <kbd>S</kbd>를 눌러 변경사항을 모두 저장하시길 바랍니다.

In [ ]:
result_dict = {
    'stage1': stage1_sub_pred.tolist(), # stage1의 예측값
    'stage2': stage2_sub_pred.tolist(), # stage2의 예측값
}

In [ ]:
import json
import os

with open("result.json", "w") as f:
    json.dump(result_dict, f)

os.system("elice_grade result.json time_series_regression.ipynb")

send files ['result.json', 'time_series_regression.ipynb'] for grade...


Compressed 2314240 bytes into 1560982 bytes ==> 67.45%


waiting result...
done!

Score: 78.241130
Duration: 2.655825138092041 seconds
=== Message ===
제출 완료되었습니다.


0